In [17]:
import cfbd
import pandas as pd
from cfbd.rest import ApiException
from datetime import datetime, timedelta

In [18]:
def setup_api():
    configuration = cfbd.Configuration()
    configuration.api_key['Authorization'] = 'aiS0bBk23RvX4eOWirh4JQgCK+UIO0k1AF/9A2RDyu2D6D/u1LLVKvdDKdHQokqF'
    configuration.api_key_prefix['Authorization'] = 'Bearer'
    client = cfbd.ApiClient(configuration)
    return {
        'games': cfbd.GamesApi(client),
        'betting': cfbd.BettingApi(client)
    }

def get_game_data(api, year, week, team):
    try:
        games = api['games'].get_games(year=year, week=week, team=team)
        if not games:
            print(f"No games found for {team} in week {week} of {year}")
            return None, None
        
        game = games[0]
        advanced_box = api['games'].get_advanced_box_score(game_id=game.id)
        return game, advanced_box
    except ApiException as e:
        print(f"Exception when calling API: {e}")
        return None, None

In [22]:
def get_current_week():
    today = datetime.now()
    season_start = datetime(today.year, 8, 1)
    week = ((today - season_start).days // 7) + 1
    return min(week, 15)

def get_game_data(api, year, week, team):
    try:
        games = api['games'].get_games(year=year, week=week, team=team)
        if not games:
            print(f"No games found for {team} in week {week} of {year}")
            return None, None
        
        game = games[0]
        advanced_box = api['games'].get_advanced_box_score(game_id=game.id)
        return game, advanced_box
    except ApiException as e:
        print(f"Exception when calling API: {e}")
        return None, None

def analyze_game(game, advanced_stats, team):
    analysis = {
        'home_team': game.home_team,
        'away_team': game.away_team,
        'date': game.start_date,
        'completed': game.completed
    }
    
    if game.completed:
        team_score = game.home_points if game.home_team == team else game.away_points
        opponent_score = game.away_points if game.home_team == team else game.home_points
        
        analysis['result'] = 'won' if team_score > opponent_score else 'lost'
        analysis['score'] = f"{team_score}-{opponent_score}"
        
        if advanced_stats:
            offense = advanced_stats.offense
            defense = advanced_stats.defense
            
            analysis.update({
                'plays': getattr(offense, 'plays', 'N/A'),
                'yards_per_play': round(getattr(offense, 'yards_per_play', 0), 2),
                'success_rate': f"{getattr(offense, 'success_rate', 0) * 100:.1f}%",
                'explosiveness': round(getattr(offense, 'explosiveness', 0), 2),
                'opponent_plays': getattr(defense, 'plays', 'N/A'),
                'opponent_yards_per_play': round(getattr(defense, 'yards_per_play', 0), 2),
                'opponent_success_rate': f"{getattr(defense, 'success_rate', 0) * 100:.1f}%",
                'opponent_explosiveness': round(getattr(defense, 'explosiveness', 0), 2),
            })
    else:
        analysis['result'] = 'upcoming'
        analysis['score'] = 'Not played yet'
    
    return analysis

def get_betting_data(api, year, week, team):
    try:
        lines = api['betting'].get_lines(year=year, week=week)
        betting_data = [
            {
                'home_team': game.home_team,
                'away_team': game.away_team,
                'provider': line.provider,
                'spread': line.spread,
                'formatted_spread': line.formatted_spread,
                'over_under': line.over_under,
                'home_moneyline': line.home_moneyline,
                'away_moneyline': line.away_moneyline,
                'game_date': game.start_date
            }
            for game in lines
            for line in game.lines
            if game.home_team == team or game.away_team == team
        ]
        return pd.DataFrame(betting_data)
    except ApiException as e:
        print(f"Exception when calling BettingApi->get_lines: {e}")
        return pd.DataFrame()

def generate_insights(game_analysis, betting_df, team):
    insights = []
    
    if game_analysis['completed']:
        insights.extend([
            f"{'Great win' if game_analysis['result'] == 'won' else 'Tough loss'} against {game_analysis['away_team'] if game_analysis['home_team'] == team else game_analysis['home_team']}",
            f"Score: {game_analysis['score']}",
        ])
        
        if 'plays' in game_analysis:
            insights.extend([
                f"Ran {game_analysis['plays']} plays vs opponent's {game_analysis['opponent_plays']}",
                f"Averaged {game_analysis['yards_per_play']} yards per play vs opponent's {game_analysis['opponent_yards_per_play']}",
                f"Offensive success rate: {game_analysis['success_rate']}",
                f"Offensive explosiveness: {game_analysis['explosiveness']}",
            ])
    else:
        insights.extend([
            f"Upcoming game against {game_analysis['away_team'] if game_analysis['home_team'] == team else game_analysis['home_team']}",
            f"Date: {game_analysis['date']}",
        ])
    
    if not betting_df.empty:
        avg_spread = betting_df['spread'].mean()
        avg_over_under = betting_df['over_under'].mean()
        insights.extend([
            f"\nBetting Insights:",
            f"Average spread: {avg_spread:.1f}",
            f"Average over/under: {avg_over_under:.1f}",
            f"{team} is the {'home' if betting_df.iloc[0]['home_team'] == team else 'away'} team",
            f"{'Favored' if (betting_df.iloc[0]['home_team'] == team) == (avg_spread < 0) else 'Underdog'} by {abs(avg_spread):.1f} points"
        ])
    
    return "\n".join(insights)


In [23]:
import matplotlib.pyplot as plt
import seaborn as sns

def create_detailed_quarter_visualizations(advanced_box, team):
    team_data = next((t for t in advanced_box.teams.ppa if t.team == team), None)
    opponent_data = next((t for t in advanced_box.teams.ppa if t.team != team), None)
    
    if not team_data or not opponent_data:
        print("Unable to create visualizations: missing data")
        return
    
    quarters = ['quarter1', 'quarter2', 'quarter3', 'quarter4']
    
    # PPA Breakdown
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 18))
    
    # Overall PPA
    team_overall = [getattr(team_data.overall, q) for q in quarters]
    opponent_overall = [getattr(opponent_data.overall, q) for q in quarters]
    ax1.bar(quarters, team_overall, alpha=0.7, label=team)
    ax1.bar(quarters, opponent_overall, alpha=0.7, label=opponent_data.team)
    ax1.set_title('Overall PPA by Quarter')
    ax1.set_ylabel('PPA')
    ax1.legend()
    
    # Passing PPA
    team_passing = [getattr(team_data.passing, q) for q in quarters]
    opponent_passing = [getattr(opponent_data.passing, q) for q in quarters]
    ax2.bar(quarters, team_passing, alpha=0.7, label=team)
    ax2.bar(quarters, opponent_passing, alpha=0.7, label=opponent_data.team)
    ax2.set_title('Passing PPA by Quarter')
    ax2.set_ylabel('PPA')
    ax2.legend()
    
    # Rushing PPA
    team_rushing = [getattr(team_data.rushing, q) for q in quarters]
    opponent_rushing = [getattr(opponent_data.rushing, q) for q in quarters]
    ax3.bar(quarters, team_rushing, alpha=0.7, label=team)
    ax3.bar(quarters, opponent_rushing, alpha=0.7, label=opponent_data.team)
    ax3.set_title('Rushing PPA by Quarter')
    ax3.set_ylabel('PPA')
    ax3.legend()
    
    plt.tight_layout()
    plt.savefig('ppa_breakdown.png')
    plt.close()
    
    # Success Rates
    team_success = next((t for t in advanced_box.teams.success_rates if t.team == team), None)
    opponent_success = next((t for t in advanced_box.teams.success_rates if t.team != team), None)
    
    if team_success and opponent_success:
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 18))
        
        # Overall Success Rate
        team_overall = [getattr(team_success.overall, q) for q in quarters]
        opponent_overall = [getattr(opponent_success.overall, q) for q in quarters]
        ax1.plot(quarters, team_overall, marker='o', label=team)
        ax1.plot(quarters, opponent_overall, marker='o', label=opponent_data.team)
        ax1.set_title('Overall Success Rate by Quarter')
        ax1.set_ylabel('Success Rate')
        ax1.legend()
        
        # Standard Downs Success Rate
        team_standard = [getattr(team_success.standard_downs, q) for q in quarters]
        opponent_standard = [getattr(opponent_success.standard_downs, q) for q in quarters]
        ax2.plot(quarters, team_standard, marker='o', label=team)
        ax2.plot(quarters, opponent_standard, marker='o', label=opponent_data.team)
        ax2.set_title('Standard Downs Success Rate by Quarter')
        ax2.set_ylabel('Success Rate')
        ax2.legend()
        
        # Passing Downs Success Rate
        team_passing = [getattr(team_success.passing_downs, q) for q in quarters]
        opponent_passing = [getattr(opponent_success.passing_downs, q) for q in quarters]
        ax3.plot(quarters, team_passing, marker='o', label=team)
        ax3.plot(quarters, opponent_passing, marker='o', label=opponent_data.team)
        ax3.set_title('Passing Downs Success Rate by Quarter')
        ax3.set_ylabel('Success Rate')
        ax3.legend()
        
        plt.tight_layout()
        plt.savefig('success_rates.png')
        plt.close()
    
    # Explosiveness
    team_explosiveness = next((t for t in advanced_box.teams.explosiveness if t.team == team), None)
    opponent_explosiveness = next((t for t in advanced_box.teams.explosiveness if t.team != team), None)
    
    if team_explosiveness and opponent_explosiveness:
        plt.figure(figsize=(10, 6))
        team_exp = [getattr(team_explosiveness.overall, q) for q in quarters]
        opponent_exp = [getattr(opponent_explosiveness.overall, q) for q in quarters]
        plt.plot(quarters, team_exp, marker='o', label=team)
        plt.plot(quarters, opponent_exp, marker='o', label=opponent_data.team)
        plt.title('Explosiveness by Quarter')
        plt.ylabel('Explosiveness')
        plt.legend()
        plt.savefig('explosiveness.png')
        plt.close()
    
    print("Detailed visualizations saved as 'ppa_breakdown.png', 'success_rates.png', and 'explosiveness.png'")

# Add this function call in the main() function of the previous script
# create_detailed_quarter_visualizations(advanced_box, team)

In [26]:
def analyze_advanced_box_score(advanced_box, team):
    if not advanced_box or not advanced_box.teams:
        return None

    team_data = next((t for t in advanced_box.teams.ppa if t.team == team), None)
    opponent_data = next((t for t in advanced_box.teams.ppa if t.team != team), None)
    
    if not team_data or not opponent_data:
        return None
    
    analysis = {
        'team': team,
        'opponent': opponent_data.team,
        'ppa': {
            'team': {
                'overall': team_data.overall.total,
                'passing': team_data.passing.total,
                'rushing': team_data.rushing.total
            },
            'opponent': {
                'overall': opponent_data.overall.total,
                'passing': opponent_data.passing.total,
                'rushing': opponent_data.rushing.total
            }
        },
        'success_rates': {
            'team': next((t.overall.total for t in advanced_box.teams.success_rates if t.team == team), None),
            'opponent': next((t.overall.total for t in advanced_box.teams.success_rates if t.team != team), None)
        },
        'explosiveness': {
            'team': next((t.overall.total for t in advanced_box.teams.explosiveness if t.team == team), None),
            'opponent': next((t.overall.total for t in advanced_box.teams.explosiveness if t.team != team), None)
        }
    }
    
    # Add rushing data
    team_rushing = next((t for t in advanced_box.teams.rushing if t.team == team), None)
    if team_rushing:
        analysis['rushing'] = {
            'power_success': team_rushing.power_success,
            'stuff_rate': team_rushing.stuff_rate,
            'line_yards': team_rushing.line_yards,
            'second_level_yards': team_rushing.second_level_yards,
            'open_field_yards': team_rushing.open_field_yards
        }
    
    # Add havoc data
    team_havoc = next((t for t in advanced_box.teams.havoc if t.team == team), None)
    if team_havoc:
        analysis['havoc'] = {
            'total': team_havoc.total,
            'front_seven': team_havoc.front_seven,
            'db': team_havoc.db
        }
    
    return analysis

# Make sure to add this function to your script, and update the main function to use it correctly

In [27]:
def main():
    api = setup_api()
    team = 'Oklahoma'
    year = 2024
    week = 5  # Adjust this based on the actual week for the Auburn game (9/28/2024)
    
    game, advanced_box = get_game_data(api, year, week, team)
    if game and advanced_box:
        analysis = analyze_advanced_box_score(advanced_box, team)
        if analysis:
            print(f"\nAdvanced Box Score Analysis for {team} vs {analysis['opponent']}:")
            print(f"PPA Overall: {team} {analysis['ppa']['team']['overall']:.2f} vs {analysis['opponent']} {analysis['ppa']['opponent']['overall']:.2f}")
            print(f"PPA Passing: {team} {analysis['ppa']['team']['passing']:.2f} vs {analysis['opponent']} {analysis['ppa']['opponent']['passing']:.2f}")
            print(f"PPA Rushing: {team} {analysis['ppa']['team']['rushing']:.2f} vs {analysis['opponent']} {analysis['ppa']['opponent']['rushing']:.2f}")
            print(f"Success Rate: {team} {analysis['success_rates']['team']:.2f} vs {analysis['opponent']} {analysis['success_rates']['opponent']:.2f}")
            print(f"Explosiveness: {team} {analysis['explosiveness']['team']:.2f} vs {analysis['opponent']} {analysis['explosiveness']['opponent']:.2f}")
            
            if 'rushing' in analysis:
                print(f"\nRushing Analysis for {team}:")
                print(f"Power Success: {analysis['rushing']['power_success']:.2f}")
                print(f"Stuff Rate: {analysis['rushing']['stuff_rate']:.2f}")
                print(f"Line Yards: {analysis['rushing']['line_yards']:.2f}")
                print(f"Second Level Yards: {analysis['rushing']['second_level_yards']:.2f}")
                print(f"Open Field Yards: {analysis['rushing']['open_field_yards']:.2f}")
            
            if 'havoc' in analysis:
                print(f"\nHavoc Rates for {team}:")
                print(f"Total: {analysis['havoc']['total']:.2f}")
                print(f"Front Seven: {analysis['havoc']['front_seven']:.2f}")
                print(f"DB: {analysis['havoc']['db']:.2f}")
            
            visualize_quarter_performance(advanced_box, team)
            print("\nQuarter-by-quarter performance visualization saved as 'quarter_performance.png'")
            
            create_detailed_quarter_visualizations(advanced_box, team)
            print("\nDetailed visualizations saved as 'ppa_breakdown.png', 'success_rates.png', and 'explosiveness.png'")
        
        betting_df = get_betting_data(api, year, week, team)
        if not betting_df.empty:
            print("\nBetting Insights:")
            avg_spread = betting_df['spread'].mean()
            avg_over_under = betting_df['over_under'].mean()
            print(f"Average spread: {avg_spread:.1f}")
            print(f"Average over/under: {avg_over_under:.1f}")
            print(f"{team} is the {'home' if betting_df.iloc[0]['home_team'] == team else 'away'} team")
            print(f"{'Favored' if (betting_df.iloc[0]['home_team'] == team) == (avg_spread < 0) else 'Underdog'} by {abs(avg_spread):.1f} points")
            
            print("\nDetailed Betting Data:")
            print(betting_df)
    else:
        print(f"No game data found for {team} in week {week} of {year}")

if __name__ == "__main__":
    main()


Betting Insights:
Average spread: 2.0
Average over/under: 45.2
Oklahoma is the away team
Favored by 2.0 points

Detailed Betting Data:
  home_team away_team    provider  spread formatted_spread  over_under  \
0    Auburn  Oklahoma    ESPN Bet     2.0      Oklahoma -2        45.5   
1    Auburn  Oklahoma  DraftKings     2.0      Oklahoma -2        45.0   
2    Auburn  Oklahoma      Bovada     2.0    Oklahoma -2.0        45.0   

   home_moneyline  away_moneyline                 game_date  
0           105.0          -125.0  2024-09-28T19:30:00.000Z  
1           110.0          -130.0  2024-09-28T19:30:00.000Z  
2           110.0          -130.0  2024-09-28T19:30:00.000Z  
